In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone.grpc import PineconeGRPC as Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains.combine_documents import create_stuff_documents_chain

In [3]:
PINECONE_API_KEY = "pcsk_4jYuMT_M8gYjRSyyUbNMcM2JVYrp4R2v3YmHE2iTaC7sEWikiPTd8JPwNRh5uYPvDZQCvW"

In [4]:
index_name = "knowledgebase"

In [5]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf("../data/")


In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [11]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\Shivraj\AppData\Local\Temp\ipykernel_21924\4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
pc = Pinecone(api_key = PINECONE_API_KEY)

In [15]:
from pinecone import Pinecone

In [16]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(index_name)

In [17]:
docsearch = PineconeVectorStore(index=index, embedding=embeddings)

In [18]:
docsearch.add_documents(documents=text_chunks)

['062c496e-fcd8-4117-8923-be94ed092707',
 '0ccc73d5-d6c4-4c01-a21c-95585937ee67',
 '28934910-8781-4930-a4b5-ae7ac39aafd5',
 'e091a6cc-dec3-4ab3-ae60-9c7c2d59aceb',
 '942b390f-eece-44eb-b833-3f1c43165d31',
 '2ffe1e2c-2dd7-4d62-9b8b-9a5dae994391',
 '7589f0cd-8706-4a89-942b-301b24ff9f83',
 '2611c317-c023-40ef-ab84-a0efc1df6066',
 '651ddebc-0f78-4532-b11b-9d2d09490260',
 'c0678f61-e5c1-4a1c-bd0f-2b3b4dc77cc7',
 '1452371f-4a31-43f7-bb29-299500aea220',
 'a03c27a4-5f73-463d-919d-32228d95b3a2',
 '6de52216-a9d8-403a-b736-6610f93c308b',
 '9d87da5d-d4c3-41b7-9227-a24b701a52ea',
 '8c603a47-5f76-4bd5-9801-2d2244338b8e',
 'b1c6778e-f528-4a96-974b-ba62551f4e07',
 '1453dd9b-50f5-4551-8f68-d236faa3eb7d',
 'f556aacd-dde7-47d8-88d5-c6bc9b004346',
 'f3363e60-7c5b-4aef-8436-5212766135ad',
 '2661f242-2717-411f-9ac0-de092f305f16',
 '9fc4f288-3f79-4a7a-9830-c8079a47707a',
 'df65a574-5d72-4ec0-bd5b-87c5dd62735c',
 '59d30ce2-d3e4-404e-91a7-0775cfb23d52',
 'e0b10d87-bd96-4f66-af0d-4196764a9747',
 '001fc470-b12f-

In [19]:
query = "What are Allergies"
docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(id='748393d6-e8c6-4d56-b5bb-5bee6144f906', metadata={'page': 135.0, 'source': '..\\data\\Medical_book.pdf'}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(id='7dd63cbf-7d5a-45ea-8d3a-72ccbab3b055', metadata={'page': 129.0, 'source': '..\\data\\Medical_book.pdf'}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in 

In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [23]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",config={'max_new_tokens':512,
                          'temperature':0.8})

In [24]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [25]:
user_input= "What is Acne"
result=qa({"query": user_input})
print("Response : ", result["result"])

C:\Users\Shivraj\AppData\Local\Temp\ipykernel_21924\857532233.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back, occurring when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
